## Imports

In [ ]:
import logging
from datetime import datetime as DT
from datetime import timedelta as TD
from dateutil import parser

from rich import print
import pandas as pd
from pandas import Series
from pandas import DataFrame as DF
import plotly.express as px

from core.s3_utils import S3_Bucket
from jobs.base_jobs.job_interval import Jobinterval
from core.constants import *
from core.time_series_processing import preprocess_date
from jobs.high_mobility.constants import *

## Setup

Lets extract just one raw ts for now. 

In [ ]:
VINS = [
    "VF1AG000065981036",
    "VF1AG000066464376",
    "VF1AG000165833302",
    "VF1AG000166152485",
    "VF1AG000166288325",
    "VF1AG000166417017",
    "VF1AG000366007352",
    "VF1AG000366046670",
    "VF1AG000464228275",
]
bucket = S3_Bucket()
def get_renault_raw_ts(vin:str) -> DF:
    return (
        bucket.read_parquet_df(f"raw_ts/renault/time_series/{vin}.parquet")
        .set_index("date", drop=False)
        .sort_index()
        .assign(soc=lambda raw_ts:raw_ts["charging.battery_level"] * 100)
    )
raw_tss:DF = {vin: get_renault_raw_ts(vin) for vin in VINS}

## Time series visualization

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(raw_tss["VF1AG000065981036"])

In [ ]:
COLS_TO_DISPALY = [
    # "charging.battery_charge_type",
    "charging.distance_to_complete_charge",
    # "charging.plugged_in",
    # "charging.status",
    # "diagnostics.odometer",
    "charging.battery_energy",
    "soc",
    "charging.estimated_range"
]
# px.line(raw_ts, COLS_TO_DISPALY)
px.line(raw_tss["VF1AG000066464376"][COLS_TO_DISPALY], )